In [1]:
import time
import pandas as pd
import numpy as np
import os
print(os.getcwd())

D:\coding\recommendation\LH_competition_recommendation\baselib\EDA


locale: the locale code of the product ['DE' 'JP' 'UK' 'ES' 'FR' 'IT']
ES:Spanish, DE:German

id: a unique for the product. Also known as Amazon Standard Item Number (ASIN) (e.g., B07WSY3MG8)

title: title of the item (e.g., “Japanese Aesthetic Sakura Flowers Vaporwave Soft Grunge Gift T-Shirt”)

price: price of the item in local currency (e.g., 24.99)

brand: item brand name (e.g., “Japanese Aesthetic Flowers & Vaporwave Clothing”)

color: color of the item (e.g., “Black”)

size: size of the item (e.g., “xxl”)

model: model of the item (e.g., “iphone 13”)

material: material of the item (e.g., “cotton”)

author: author of the item (e.g., “J. K. Rowling”)

desc: description about a item’s key features and benefits called out via bullet points (e.g., “Solid colors: 100% Cotton; Heather Grey: 90% Cotton, 10% Polyester; All Other Heathers …”)

In [2]:
data_root = 'D:/data/KDD2023'

productfile = pd.read_csv(os.path.join(data_root, 'products_train.csv'))  # 商品信息
userfile = pd.read_csv(os.path.join(data_root, 'sessions_train.csv'))  # 用户信息
task = 'task1'
test_data = pd.read_csv(os.path.join(data_root, f'sessions_test_{task}.csv'))

In [3]:
productfile.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [4]:
userfile.head()

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


In [5]:
test_data.head()

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE


In [6]:
def read_locale_data(locale):
    products = productfile.query(f'locale == "{locale}"')
    sess_train = userfile.query(f'locale == "{locale}"')
    sess_test = test_data.query(f'locale == "{locale}"')
    return products, sess_train, sess_test

def show_locale_info(locale):
    products, sess_train, sess_test = read_locale_data(locale)

    train_l = sess_train['prev_items'].apply(lambda sess: len(sess.replace("'", "").strip("[]").split(' ')))
    test_l = sess_test['prev_items'].apply(lambda sess: len(sess.replace("'", "").strip("[]").split(' ')))

    print(f"Locale: {locale} \n"
          f"Number of products: {products['id'].nunique()} \n"
          f"Number of train sessions: {len(sess_train)} \n"
          f"Train session lengths - "
          f"Mean: {train_l.mean():.2f} | Median {train_l.median():.2f} | "
          f"Min: {train_l.min():.2f} | Max {train_l.max():.2f} \n"
          f"Number of test sessions: {len(sess_test)}"
        )
    if len(sess_test) > 0:
        print(
             f"Test session lengths - "
            f"Mean: {test_l.mean():.2f} | Median {test_l.median():.2f} | "
            f"Min: {test_l.min():.2f} | Max {test_l.max():.2f} \n"
        )
    print("======================================================================== \n")

In [7]:

locale_names = productfile['locale'].unique()
print(locale_names)
for locale in locale_names:
    show_locale_info(locale)

['DE' 'JP' 'UK' 'ES' 'FR' 'IT']
Locale: DE 
Number of products: 518327 
Number of train sessions: 1111416 
Train session lengths - Mean: 4.35 | Median 3.00 | Min: 2.00 | Max 156.00 
Number of test sessions: 104568
Test session lengths - Mean: 4.30 | Median 3.00 | Min: 2.00 | Max 53.00 


Locale: JP 
Number of products: 395009 
Number of train sessions: 979119 
Train session lengths - Mean: 4.48 | Median 3.00 | Min: 2.00 | Max 474.00 
Number of test sessions: 96467
Test session lengths - Mean: 4.51 | Median 3.00 | Min: 2.00 | Max 112.00 


Locale: UK 
Number of products: 500180 
Number of train sessions: 1182181 
Train session lengths - Mean: 4.12 | Median 3.00 | Min: 2.00 | Max 201.00 
Number of test sessions: 115936
Test session lengths - Mean: 4.02 | Median 3.00 | Min: 2.00 | Max 66.00 


Locale: ES 
Number of products: 42503 
Number of train sessions: 89047 
Train session lengths - Mean: 3.66 | Median 3.00 | Min: 2.00 | Max 60.00 
Number of test sessions: 0

Locale: FR 
Number of pr

In [8]:
PREDS_PER_SESSION = 1000
def random_predicitons(locale, sess_test_locale):
    random_state = np.random.RandomState(42)
    products = productfile.query(f'locale == "{locale}"')
    predictions = []
    for _ in range(len(sess_test_locale)):
        predictions.append(
            list(products['id'].sample(PREDS_PER_SESSION, replace=True, random_state=random_state))
        )
    sess_test_locale['next_item_prediction'] = predictions
    sess_test_locale.drop('prev_items', inplace=True, axis=1)
    return sess_test_locale


In [11]:
predictions = []
test_locale_names = test_data['locale'].unique()
for locale in test_locale_names:
    sess_test_locale = test_data.query(f'locale == "{locale}"').copy()
    predictions.append(
        random_predicitons(locale, sess_test_locale)
    )
predictions = pd.concat(predictions).reset_index(drop=True)
print(predictions.shape)
predictions.sample(5)

(316971, 2)


,locale,next_item_prediction
14069,DE,"[B08T1LP7KH, B01LZ5A6F3, 3772444571, B09K7F2N7..."
224017,UK,"[B09JWLR3RS, B0074O8YYY, B076932C58, B01ENJ1WF..."
75917,DE,"[B08XZQKNW9, B0BFP1BXM4, B0B2RQCJ43, B09M74DQG..."
184045,JP,"[B0B8MTMHJH, B0BK8X3786, B07NFCVCPP, B004O6KX4..."
104154,DE,"[B09LDCTHXW, B0992NL6CH, B01M3RG1VR, 355105171..."


In [ ]:
def check_predictions(predictions, check_products=False):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = test_data['locale'].unique()
    for locale in test_locale_names:
        sess_test = test_data.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"

        if check_products:
            # This check is not done on the evaluator
            # but you can run it to verify there is no mixing of products between locales
            # Since the ground truth next item will always belong to the same locale
            # Warning - This can be slow to run
            products = productfile.query(f'locale == "{locale}"')
            predicted_products = np.unique( np.array(list(preds_locale["next_item_prediction"].values)) )
            assert np.all( np.isin(predicted_products, products['id']) ), f"Invalid products in {locale} predictions"
    print('fine')

In [ ]:
check_predictions(predictions)
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(f'./output/submission_{task}.parquet', engine='pyarrow')



In [51]:
# aicrowd submission create -c task-1-next-product-recommendation -f "submission_task1.parquet"